In [26]:
import logging
import os
import sys
import warnings
from functools import partial

import astropy.units as u
import lal.series
import numpy as np
import requests
from astropy import cosmology, units
from astropy.cosmology._utils import vectorize_redshift_method
from astropy.table import Table
from astropy.units import dimensionless_unscaled
from ligo.lw import ligolw
from ligo.lw import utils as ligolw_utils
from ligo.skymap.bayestar.filter import sngl_inspiral_psd
from ligo.skymap.util import progress_map
from scipy import integrate, optimize, special, stats
from scipy.integrate import fixed_quad, quad
from scipy.interpolate import interp1d
from scipy.optimize import root_scalar

logging.basicConfig(level=logging.INFO)

# Download and prepare GWTC-3 distribustion as farah.h5 file 

In [27]:
def download_file(url, filename):
    """Download a file from a URL and save it locally."""
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(filename, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return filename
    except Exception as e:
        print(f"Error downloading file: {e}")
        return None

### Define Paths and Filenames

In [28]:
output_dir = "../../scenarios"
farah_file = os.path.join(output_dir, "farah.h5")

## Download the File if Needed

Download the GWTC-3 PDB file (see https://dcc.ligo.org/LIGO-T2100512/public/)

In [29]:
input_file = None
if not os.path.exists(farah_file):
    file_url = "https://dcc.ligo.org/LIGO-T2100512/public/O1O2O3all_mass_h_iid_mag_iid_tilt_powerlaw_redshift_maxP_events_all.h5"
    file_name = os.path.join(output_dir, file_url.split("/")[-1])
    input_file = download_file(file_url, file_name)

    if input_file is None:
        logging.error("Failed to download the required file.")
        sys.exit(1)

## Read, Transform, and Save Data

In [30]:
if not os.path.exists(farah_file):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        # Read the original file and save only the relevant columns
        data = Table.read()
        Table(
            {
                "mass1": data["mass_1"],
                "mass2": data["mass_2"],
                "spin1z": data["a_1"] * data["cos_tilt_1"],
                "spin2z": data["a_2"] * data["cos_tilt_2"],
            }
        ).write(farah_file, overwrite=True)

else:
    print(f"farah.h5 already exists at: {farah_file}")

farah.h5 already exists at: ../../scenarios/farah.h5


## Mass threshold for neutron stars

In [31]:
ns_mass = 3.0  # Solar masses

## Clean Up and Print Statistics

In [32]:
# Remove the original large file if it exists
if input_file is not None and os.path.exists(input_file):
    os.remove(input_file)
    logging.info(f"Removed temporary file: {input_file}")

# Reload the processed file
if os.path.exists(farah_file):
    data = Table.read(farah_file)

    # Compute statistics for binary systems
    n_bns = len(data[data["mass1"] < ns_mass])
    n_nsbh = len(data[(data["mass1"] >= ns_mass) & (data["mass2"] < ns_mass)])
    n_bbh = len(data[data["mass2"] >= ns_mass])

    output = (
        "Summary of Compact Binary Types\n"
        "---------------------------------\n"
        f"Number of BNS   : {n_bns}\n"
        f"Number of NSBH  : {n_nsbh}\n"
        f"Number of BBH   : {n_bbh}\n"
        "---------------------------------\n"
    )
    print(output)
else:
    print("farah.h5 not found. Please check the previous steps.")

Summary of Compact Binary Types
---------------------------------
Number of BNS   : 892762
Number of NSBH  : 35962
Number of BBH   : 71276
---------------------------------



## SNR Selection Helper

In [33]:
def get_decisive_snr(snrs, min_triggers):
    """
    Return the SNR value that decides if an event is detectable (the min_triggers-th highest SNR).

    Parameters
    ----------
    snrs : list or array-like
        List of SNRs (floats).
    min_triggers : int
        Minimum number of triggers to form a coincidence.

    Returns
    -------
    decisive_snr : float
        The SNR of the trigger that decides detectability.
    """
    return sorted(snrs)[-min_triggers]

## Find First/Last Nonzero

In [34]:
def lo_hi_nonzero(x):
    """Return indices of the first and last nonzero elements of an array."""
    nonzero = np.flatnonzero(x)
    return nonzero[0], nonzero[-1]

## GWCosmo Class: Gravitational Wave Cosmology Tools

The `GWCosmo` class provides methods to calculate important cosmological figures of merit for gravitational wave (GW) astronomy. 
Given a cosmological model, it can estimate the maximum observable distance for GW sources, the sensitive volume out to a given redshift, and related quantities.

### Main Methods

- `z_at_snr(...)`:  
  Computes the redshift at which a GW source achieves a specific signal-to-noise ratio (SNR) in a detector network.

- `get_max_z(...)`:  
  Returns the maximum redshift (distance) at which a source can be detected, given the detector sensitivity and source parameters.

- `sensitive_volume(z)`:  
  Calculates the sensitive comoving volume out to redshift `z`, useful for GW event rate calculations.

- `sensitive_distance(z)`:  
  Returns the “sensitive distance,” defined such that the sensitive volume equals that of a sphere with radius `d_s(z)`.

In [35]:
class GWCosmo:
    """Evaluate GW distance figures of merit for a given cosmology.

    Parameters
    ----------
    cosmo : :class:`astropy.cosmology.FLRW`
        The cosmological model.

    """

    def __init__(self, cosmology):
        self.cosmo = cosmology

    def z_at_snr(
        self,
        psds,
        waveform,
        f_low,
        snr_threshold,
        min_triggers,
        mass1,
        mass2,
        spin1z,
        spin2z,
    ):
        """
        Get redshift at which a waveform attains a given SNR.

        Parameters
        ----------
        psds : list
            List of :class:`lal.REAL8FrequencySeries` objects.
        waveform : str
            Waveform approximant name.
        f_low : float
            Low-frequency cutoff for template.
        snr_threshold : float
            Minimum single-detector SNR.
        min_triggers : int
            Minimum number of triggers to form a coincidence.
        params : list
            List of waveform parameters: mass1, mass2, spin1z, spin2z.

        Returns
        -------
        comoving_distance : float
            Comoving distance in Mpc.

        """
        # Construct waveform
        series = sngl_inspiral_psd(
            waveform,
            f_low=f_low,
            mass1=mass1,
            mass2=mass2,
            spin1z=spin1z,
            spin2z=spin2z,
        )
        i_lo, i_hi = lo_hi_nonzero(series.data.data)
        log_f = np.log(series.f0 + series.deltaF * np.arange(i_lo, i_hi + 1))
        log_f_lo = log_f[0]
        log_f_hi = log_f[-1]
        num = interp1d(
            log_f,
            np.log(series.data.data[i_lo : i_hi + 1]),
            fill_value=-np.inf,
            bounds_error=False,
            assume_sorted=True,
        )

        denoms = []
        for series in psds:
            i_lo, i_hi = lo_hi_nonzero(
                np.isfinite(series.data.data) & (series.data.data != 0)
            )
            log_f = np.log(series.f0 + series.deltaF * np.arange(i_lo, i_hi + 1))
            denom = interp1d(
                log_f,
                log_f - np.log(series.data.data[i_lo : i_hi + 1]),
                fill_value=-np.inf,
                bounds_error=False,
                assume_sorted=True,
            )
            denoms.append(denom)

        def snr_at_z(z):
            logzp1 = np.log(z + 1)

            def integrand(log_f):
                return [np.exp(num(log_f + logzp1) + denom(log_f)) for denom in denoms]

            integrals, _ = fixed_quad(integrand, log_f_lo, log_f_hi - logzp1, n=1024)
            snr = get_decisive_snr(np.sqrt(4 * integrals), min_triggers)
            with np.errstate(divide="ignore"):
                snr /= self.cosmo.angular_diameter_distance(z).to_value(units.Mpc)
            return snr

        def root_func(z):
            return snr_at_z(z) - snr_threshold

        return root_scalar(root_func, bracket=(0, 1e3)).root

    def get_max_z(
        self,
        psds,
        waveform,
        f_low,
        snr_threshold,
        min_triggers,
        mass1,
        mass2,
        spin1z,
        spin2z,
        jobs=1,
    ):
        # Calculate the maximum distance on the grid.
        params = [mass1, mass2, spin1z, spin2z]
        shape = np.broadcast_shapes(*(param.shape for param in params))
        result = list(
            progress_map(
                partial(
                    self.z_at_snr, psds, waveform, f_low, snr_threshold, min_triggers
                ),
                *(param.ravel() for param in params),
                jobs=jobs,
            )
        )
        result = np.reshape(result, shape)

        assert np.all(result >= 0), "some redshifts are negative"
        assert np.all(np.isfinite(result)), "some redshifts are not finite"
        return result

    @vectorize_redshift_method
    def _sensitive_volume_integral(self, z):
        dh3_sr = self.cosmo.hubble_distance**3 / units.sr

        def integrand(z):
            result = self.cosmo.differential_comoving_volume(z)
            result /= (1 + z) * dh3_sr
            return result.to_value(dimensionless_unscaled)

        result, _ = quad(integrand, 0, z)
        return result

    def sensitive_volume(self, z):
        """Sensitive volume :math:`V(z)` out to redshift :math:`z`.

        Given a population of events that occur at a constant rate density
        :math:`R` per unit comoving volume per unit proper time, the number of
        observed events out to a redshift :math:`N(z)` over an observation time
        :math:`T` is :math:`N(z) = R T V(z)`.
        """
        dh3 = self.cosmo.hubble_distance**3
        return 4 * np.pi * dh3 * self._sensitive_volume_integral(z)

    def sensitive_distance(self, z):
        r"""Sensitive distance as a function of redshift :math:`z`.

        The sensitive distance is the distance :math:`d_s(z)` defined such that
        :math:`V(z) = 4/3\pi {d_s(z)}^3`, where :math:`V(z)` is the sensitive
        volume.
        """
        dh = self.cosmo.hubble_distance
        return dh * np.cbrt(3 * self._sensitive_volume_integral(z))

    # ----------------- Execution & Example -----------------

##### 1. We load the noise Power Spectral Densities (PSDs) from the XML file.
#####    These PSDs are used to compute signal-to-noise ratios (SNRs) across detectors.

##### Initialize a new LIGO Light-Weight XML document


In [36]:
xmldoc = ligolw.Document()
xmlroot = xmldoc.appendChild(ligolw.LIGO_LW())

#### Initialize a new LIGO Light-Weight XML document

In [37]:
reference_psd = "../../scenarios/psds.xml"
with open(reference_psd, "rb") as f:
    xmldoc = ligolw_utils.load_fileobj(f, contenthandler=lal.series.PSDContentHandler)
    psds = list(lal.series.read_psd_xmldoc(xmldoc).values())

psds

[<Swig Object of type 'tagREAL8FrequencySeries *' at 0x19dd3b130>,
 <Swig Object of type 'tagREAL8FrequencySeries *' at 0x19dd38f30>,
 <Swig Object of type 'tagREAL8FrequencySeries *' at 0x19dd382b0>,
 <Swig Object of type 'tagREAL8FrequencySeries *' at 0x19dd3afb0>]

#### 2. We define the waveform model, the low-frequency cutoff, and the detection thresholds.

In [38]:
waveform = "IMRPhenomD"  # Chosen waveform approximant
f_low = 25.0  # Low-frequency cutoff in Hz
snr_threshold = 1  # SNR threshold for detection
min_triggers = 1  # Minimum number of triggers (coincident detectors)
jobs = 1  # Number of parallel jobs


##### 3. We initialize the GWCosmo object with a cosmological model (Planck15 here).
#####    This allows us to compute distances and volumes in a cosmological framework.

In [39]:
gwcosmo = GWCosmo(getattr(cosmology, "Planck15"))



##### 4. We load a subset (first 5) of the binary neutron star / black hole samples 
#####    from an HDF5 file (`farah.h5`). Each sample includes mass1, mass2, spin1z, spin2z.

In [40]:
distribution_samples = "../../scenarios/farah.h5"
samples = Table.read(distribution_samples)[0:5]
samples

mass1,mass2,spin1z,spin2z
float64,float64,float64,float64
1.6334784673051967,1.3810887548655666,0.0005726141112029242,-0.012026548338777296
2.4116235511053596,2.038928316809099,0.09564577831587825,0.12876640961108698
2.3446204414808824,2.1620985552733023,-0.0424013188343698,0.20299660662345526
1.5224413917381097,1.4464455008707857,0.056652858388635034,0.1648556391943954
2.071907865468317,1.757569633659934,0.0721102841575518,-0.06863285853206544


### 5. Compute the Maximum Redshift (`max_z`) for Each Sample

For each sample, we compute the maximum redshift `max_z` at which the SNR exceeds the chosen threshold, using all available PSDs.
This approach uses the entire PSD frequency range to ensure accuracy.

In [41]:
max_z = gwcosmo.get_max_z(
    psds,
    waveform,
    f_low,
    snr_threshold,
    min_triggers,
    samples["mass1"],
    samples["mass2"],
    samples["spin1z"],
    samples["spin2z"],
    jobs=jobs,
)

max_z

  0%|          | 0/5 [00:00<?, ?it/s]

INFO:BAYESTAR:Selected template: IMRPhenomD
INFO:BAYESTAR:Selected template: IMRPhenomD
INFO:BAYESTAR:Selected template: IMRPhenomD
INFO:BAYESTAR:Selected template: IMRPhenomD
INFO:BAYESTAR:Selected template: IMRPhenomD


array([1.90879562, 3.01082865, 3.05186255, 1.90409028, 2.49130032])

### 6. Convert `max_z` to Comoving Sensitive Distance

We convert each value of `max_z` to a comoving sensitive distance (in Mpc) using the cosmological model.
Because `max_z` is a vector, the conversion must be applied to each element separately
to avoid errors with array-valued integration.


In [42]:
max_distance = gwcosmo.sensitive_distance(max_z).to_value(units.Mpc)
max_distance

array([3998.333539  , 4707.80752121, 4726.72584292, 3994.16514986,
       4429.10918943])

### 7. Compute Detection Volumes and Selection Probabilities

- For each sample, use the sensitive distance to calculate the corresponding detection volume:
  
  $$
  V = \frac{4}{3}\pi \times (\text{distance})^3
  $$

- These volumes are proportional to each sample's detection probability.
- Finally, for each sample, calculate the product `V * T`, where `T` is the observation time.

In [43]:
# uniform weight per sample, each distance contributes equally at first.
probs = 1 / len(max_distance)

### From Equal to Volume-Weighted Probabilities

- Instead of giving all distances equal probability, we weight each distance by the amount of observable space (volume) at that distance.

- Because sources are uniformly distributed in space, a larger volume means a higher chance of containing a source.
- Thus, the probability of detecting a source increases with the volume we can observe.

In [44]:
probs *= 4 / 3 * np.pi * max_distance**3
print(probs)

[5.35495302e+10 8.74127333e+10 8.84707777e+10 5.33822235e+10
 7.27893374e+10]


### Step: Normalize the Probabilities

To obtain a valid probability distribution:

- We divide each probability (or volume) by the total sum, so that the sum of all probabilities is exactly 1.

- This way, the probabilities represent a proper distribution that can be used to randomly draw synthetic events according to their likelihood.



In [45]:
# Normalize the probabilities so they sum to 1
volume_sum = probs.sum()
print("Sum of probabilities (before normalization):", volume_sum)

probs /= volume_sum  # Element-wise division

print("\nNormalized probabilities:\n", probs)
print("\nSum of normalized probabilities:", probs.sum())

Sum of probabilities (before normalization): 355604601947.13995

Normalized probabilities:
 [0.15058728 0.2458144  0.24878974 0.1501168  0.20469178]

Sum of normalized probabilities: 1.0


#### 8. Simulate events by drawing samples according to their normalized probabilities, using either weighted sampling or a weighted random distribution.

In [46]:
nsamples = len(samples)
dist = stats.rv_discrete(values=(np.arange(len(probs)), probs))

In [47]:
n_batches = max(nsamples * len(probs) // 1_000_000_000, 1)
batch_sizes = [
    len(subarray) for subarray in np.array_split(np.empty(nsamples), n_batches)
]

#### 9. Now, using `dist.rvs()`, we can randomly sample event indices:
Indices corresponding to higher probabilities are more likely to be chosen.

In [48]:
indices = np.concatenate([dist.rvs(size=batch_size) for batch_size in batch_sizes])
indices

array([1, 1, 1, 4, 1])

### 10.  The `cols` dictionary now contains only the selected properties (`mass1`, `mass2`, `spin1z`, `spin2z`) for the sampled events. Each entry is an array of values corresponding to your randomly drawn sample indices.

In [49]:
# Select columns for the sampled events, keeping only the desired keys
import pandas as pd

cols = {key: samples[key][indices] for key in ["mass1", "mass2", "spin1z", "spin2z"]}
pd.DataFrame(cols)  # Display the dictionary of selected columns

INFO:numexpr.utils:NumExpr defaulting to 12 threads.


,mass1,mass2,spin1z,spin2z
0,2.411624,2.038928,0.095646,0.128766
1,2.411624,2.038928,0.095646,0.128766
2,2.411624,2.038928,0.095646,0.128766
3,2.071908,1.757570,0.072110,-0.068633
4,2.411624,2.038928,0.095646,0.128766


#### Calculate the Volumetric Rate

We estimate the volumetric event rate by dividing the number of simulated events by the total sensitive volume,
and express the result in units of events per year per cubic megaparsec (yr⁻¹ Mpc⁻³).

In [50]:
volumetric_rate = nsamples / volume_sum * units.year**-1 * units.Mpc**-3
volumetric_rate

<Quantity 1.40605604e-11 1 / (yr Mpc3)>

In [51]:
# Draw random extrinsic parameters
distance = stats.powerlaw(a=3, scale=max_distance[indices]).rvs(size=nsamples)
distance

array([2994.45759318, 3469.51154598, 4102.32675857,  734.82961419,
       4522.71971105])

# Processing with All Simulations

After processing all the simulations, we will consider the new observing scenarios based on the GWTC-3 catalog:

## We need to standardize our rate base on the initial distribution rate 

In [52]:
# Lower 5% and upper 95% quantiles of log-normal distribution for different CBC populations
run_names = ["O4", "O5"]
rates_table = Table(
    [
        # Quantiles from O3 R&P paper Table II, row 1, last column
        {"population": "BNS", "lower": 100.0, "mid": 240.0, "upper": 510.0},
        {"population": "NSBH", "lower": 100.0, "mid": 240.0, "upper": 510.0},
        {"population": "BBH", "lower": 100.0, "mid": 240.0, "upper": 510.0},
    ]
)

rates_table

population,lower,mid,upper
str4,float64,float64,float64
BNS,100.0,240.0,510.0
NSBH,100.0,240.0,510.0
BBH,100.0,240.0,510.0


#### Splitting Compact Binary Populations

To classify events as BNS (Binary Neutron Star), NSBH (Neutron Star–Black Hole), or BBH (Binary Black Hole),  
we use an upper/lower mass limit of 3 solar masses to distinguish neutron stars (NS) from black holes (BH).


In [53]:
ns_max_mass = 3.0

#### Injection Summary

We injected a total of **1 million** compact binary coalescences (CBCs), consisting of:
- **892,762** binary neutron stars (BNS)
- **35,962** neutron star–black hole binaries (NSBH)
- **71,276** binary black holes (BBH)

In [54]:
# Calculate the mass fraction for each CBC population (BNS, NSBH, BBH)
# by dividing the population count by the total number of CBCs (1 million)
rates_table["mass_fraction"] = (
    np.array([892762, 35962, 71276]) / 1e6
)  # Total CBCs = 1 million


rates_table

population,lower,mid,upper,mass_fraction
str4,float64,float64,float64,float64
BNS,100.0,240.0,510.0,0.892762
NSBH,100.0,240.0,510.0,0.035962
BBH,100.0,240.0,510.0,0.071276


##### === Simulated BNS Merger Rates ===
In observing scenarios data there is a file sqlite  file "sqlite3 events.sqlite" where we can read the 
the simulation Merger rate, for that use this command line in terminal or import sqlite with python 

 Use this command to retrieve comments:

 1- $ sqlite3 events.sqlite

 2- $ select comment from process;

For example: The simulated CBC merger rate in yr^-1 Gpc^-3 for O5 and O6-HLVK configuration with (SNR = 10)

From kiendrebeogo et al. 2023 the simulated rate is given by the by (yr^-1 Mpc^-3)

so this need to be convert in yr^-1 Gpc^-3, before add use it here.

simulation rate  sim_rate =2.712359951521142e3 (u.Gpc**-3 * u.yr**-1) 

### === Simulated BNS Merger Rates ===

In the observing scenario data, there is an SQLite file called `events.sqlite` containing the simulated merger rates.

You can access the merger rate comments using either the command line or Python:

**Command-line (SQLite):**
1. Open the database:

    $ sqlite3 events.sqlite

2. Retrieve the merger rate comments:

    sqlite> select comment from process;


For example: The simulated CBC merger rate in yr^-1 Gpc^-3 for O5 and O6-HLVK configuration with (SNR = 10)

From kiendrebeogo et al. 2023 the simulated rate is given by the by (yr^-1 Mpc^-3)

so this need to be convert in yr^-1 Gpc^-3, before add use it here.

In [55]:
# Here BNS, BBH and NSBH have the same simulation rates as they have been simulated together.
rates_table["sim_rate_O4"] = [6.5462076e-06, 6.5462076e-06, 6.5462076e-06] * (
    1 / (u.Mpc**3 * u.yr)
)
rates_table["sim_rate_O5"] = [2.71235995e-06, 2.71235995e-06, 2.71235995e-06] * (
    1 / (u.Mpc**3 * u.yr)
)

# Conversion in Gpc^-3/ yr
rates_table["sim_rate_O4"] = rates_table["sim_rate_O4"].to(u.Gpc**-3 * u.yr**-1)
rates_table["sim_rate_O5"] = rates_table["sim_rate_O5"].to(u.Gpc**-3 * u.yr**-1)

rates_table["sim_rate_O4"], rates_table["sim_rate_O5"]

(<Column name='sim_rate_O4' dtype='float64' unit='1 / (yr Gpc3)' length=3>
 6546.207600000001
 6546.207600000001
 6546.207600000001,
 <Column name='sim_rate_O5' dtype='float64' unit='1 / (yr Gpc3)' length=3>
 2712.35995
 2712.35995
 2712.35995)

# Add the expected number of detections for O4 and O5

Here the SNR threshold to confirm a detection is 8.

So all those event have a signal noise ratio > 8

In [56]:
rates_table["detection_number_O4"] = np.array([1004, 184, 7070])
rates_table["detection_number_O5"] = np.array([2003, 356, 9809])

rates_table

population,lower,mid,upper,mass_fraction,sim_rate_O4,sim_rate_O5,detection_number_O4,detection_number_O5
,,,,,1 / (yr Gpc3),1 / (yr Gpc3),,
str4,float64,float64,float64,float64,float64,float64,int64,int64
BNS,100.0,240.0,510.0,0.892762,6546.207600000001,2712.35995,1004,2003
NSBH,100.0,240.0,510.0,0.035962,6546.207600000001,2712.35995,184,356
BBH,100.0,240.0,510.0,0.071276,6546.207600000001,2712.35995,7070,9809


# Scaling Quantiles by Mass Fraction

For each CBC population, we scale the lower, median, and upper event rate quantiles by `mass_fraction` column. 

This can be used, for example, to focus on a sub-population or a fraction of events relevant to a specific analysis.

> **Note:** Make sure that the `mass_fraction` column exists and has appropriate values (between 0 and 1).


In [57]:
for key in ["lower", "mid", "upper"]:
    rates_table[key] *= rates_table["mass_fraction"]

rates_table

population,lower,mid,upper,mass_fraction,sim_rate_O4,sim_rate_O5,detection_number_O4,detection_number_O5
,,,,,1 / (yr Gpc3),1 / (yr Gpc3),,
str4,float64,float64,float64,float64,float64,float64,int64,int64
BNS,89.27619999999999,214.26288,455.30861999999996,0.892762,6546.207600000001,2712.35995,1004,2003
NSBH,3.5962,8.63088,18.34062,0.035962,6546.207600000001,2712.35995,184,356
BBH,7.127600000000001,17.10624,36.35076,0.071276,6546.207600000001,2712.35995,7070,9809


## Compute Log-normal Parameters from Quantiles

For each population, we calculate the mean (`mu`) and standard deviation (`sigma`) of the underlying log-normal distribution.
- `mu` is the mean of the logarithm of the median event rate.
- `sigma` is set so that the interval between the lower and upper quantile matches the standard width of a 90% normal interval (from 5% to 95% quantile).

This is useful for statistical modeling or simulations where log-normal priors are assumed for event rates.


In [58]:
(standard_90pct_interval,) = np.diff(stats.norm.interval(0.9))
rates_table["mu"] = np.log(rates_table["mid"])
rates_table["sigma"] = (
    np.log(rates_table["upper"]) - np.log(rates_table["lower"])
) / standard_90pct_interval

rates_table

population,lower,mid,upper,mass_fraction,sim_rate_O4,sim_rate_O5,detection_number_O4,detection_number_O5,mu,sigma
,,,,,1 / (yr Gpc3),1 / (yr Gpc3),,,,
str4,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64
BNS,89.27619999999999,214.26288,455.30861999999996,0.892762,6546.207600000001,2712.35995,1004,2003,5.367203672357068,0.49525395847832065
NSBH,3.5962,8.63088,18.34062,0.035962,6546.207600000001,2712.35995,184,356,2.1553464697693,0.49525395847832093
BBH,7.127600000000001,17.10624,36.35076,0.071276,6546.207600000001,2712.35995,7070,9809,2.8394433092250226,0.4952539584783208


In [59]:
# Extract the log-normal parameters for each population as numpy arrays
fiducial_log_rates = np.asarray(rates_table["mu"])
fiducial_log_rate_errs = np.asarray(rates_table["sigma"])

fiducial_log_rates, fiducial_log_rate_errs

(array([5.36720367, 2.15534647, 2.83944331]),
 array([0.49525396, 0.49525396, 0.49525396]))

### Add Log-normal Parameters to the Table

For convenience, we add the log-normal mean (`fiducial_log_rate`) and standard deviation (`fiducial_log_rate_err`) as new columns in the `rates_table`.  
This keeps all relevant parameters together and makes the table easy to use for further analysis or plotting.


In [60]:
rates_table["fiducial_log_rate"] = fiducial_log_rates
rates_table["fiducial_log_rate_err"] = fiducial_log_rate_errs

rates_table

population,lower,mid,upper,mass_fraction,sim_rate_O4,sim_rate_O5,detection_number_O4,detection_number_O5,mu,sigma,fiducial_log_rate,fiducial_log_rate_err
,,,,,1 / (yr Gpc3),1 / (yr Gpc3),,,,,,
str4,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64
BNS,89.27619999999999,214.26288,455.30861999999996,0.892762,6546.207600000001,2712.35995,1004,2003,5.367203672357068,0.49525395847832065,5.367203672357068,0.49525395847832065
NSBH,3.5962,8.63088,18.34062,0.035962,6546.207600000001,2712.35995,184,356,2.1553464697693,0.49525395847832093,2.1553464697693,0.49525395847832093
BBH,7.127600000000001,17.10624,36.35076,0.071276,6546.207600000001,2712.35995,7070,9809,2.8394433092250226,0.4952539584783208,2.8394433092250226,0.4952539584783208


#### Functions for propagating errors in rates

Reproduced from https://github.com/lpsinger/observing-scenarios-simulations/blob/main/plots-and-tables.ipynb.

In [61]:
def betabinom_k_n(k, n):
    return stats.betabinom(n, k + 1, n - k + 1)


@np.vectorize
def poisson_lognormal_rate_cdf(k, mu, sigma):
    lognorm_pdf = stats.lognorm(s=sigma, scale=np.exp(mu)).pdf

    def func(lam):
        prior = lognorm_pdf(lam)
        # poisson_pdf = np.exp(special.xlogy(k, lam) - special.gammaln(k + 1) - lam)
        poisson_cdf = special.gammaincc(k + 1, lam)
        return poisson_cdf * prior

    # Marginalize over lambda.
    #
    # Note that we use scipy.integrate.odeint instead
    # of scipy.integrate.quad because it is important for the stability of
    # root_scalar below that we calculate the pdf and the cdf at the same time,
    # using the same exact quadrature rule.
    cdf, _ = integrate.quad(func, 0, np.inf, epsabs=0)
    return cdf


@np.vectorize
def poisson_lognormal_rate_quantiles(p, mu, sigma):
    """Find the quantiles of a Poisson distribution with
    a log-normal prior on its rate.

    Parameters
    ----------
    p : float
        The quantiles at which to find the number of counts.
    mu : float
        The mean of the log of the rate.
    sigma : float
        The standard deviation of the log of the rate.

    Returns
    -------
    k : float
        The number of events.

    Notes
    -----
    This algorithm treats the Poisson count k as a continuous
    real variable so that it can use the scipy.optimize.root_scalar
    root finding/polishing algorithms.
    """

    def func(k):
        return poisson_lognormal_rate_cdf(k, mu, sigma) - p

    if func(0) >= 0:
        return 0

    result = optimize.root_scalar(func, bracket=[0, 1e6])
    return result.root


def format_with_errorbars(mid, lo, hi):
    plus = hi - mid
    minus = mid - lo
    smallest = min(max(0, plus), max(0, minus))

    if smallest == 0:
        return str(mid), "0", "0"
    decimals = 1 - int(np.floor(np.log10(smallest)))

    if all(np.issubdtype(type(_), np.integer) for _ in (mid, lo, hi)):
        decimals = min(decimals, 0)

    plus, minus, mid = np.round([plus, minus, mid], decimals)
    if decimals > 0:
        fstring = "%%.0%df" % decimals
    else:
        fstring = "%d"
    return [fstring % _ for _ in [mid, minus, plus]]

### Set Probability Quantiles and Run Duration

- `prob_quantiles` defines the lower (5%), median (50%), and upper (95%) quantiles used for statistical calculations.
- `run_duration` is the duration of the observing run in years.  
You can adjust this value if you need rates for a different observation period.


In [62]:
prob_quantiles = np.asarray([0.05, 0.5, 0.95])
run_duration = 1.0  # years

### Detection Number Quantiles Table

This block computes the 5%, 50%, and 95% quantiles for the expected number of detections for each CBC population and run, based on log-normal statistics.

**Table columns:**
- `run`: Observing run (e.g., O4, O5)
- `population`: Source type (BNS, NSBH, BBH)
- `lo`, `mid`, `hi`: Lower, median, and upper quantiles for the expected number of detections


In [63]:
results = {}
for run_name in run_names:
    results[run_name] = {}

    for pop in ["BNS", "NSBH", "BBH"]:
        rates_row = rates_table[rates_table["population"] == pop]
        rate = rates_row[f"sim_rate_{run_name}"] * rates_row["mass_fraction"]

        mu = (
            rates_row["fiducial_log_rate"]
            + np.log(run_duration)
            + np.log(rates_row[f"detection_number_{run_name}"] / rate)
        )
        sigma = rates_row["fiducial_log_rate_err"]

        lo, mid, hi = poisson_lognormal_rate_quantiles(prob_quantiles, mu, sigma)
        print(lo, mid, hi)
        lo = int(np.floor(lo))
        mid = int(np.round(mid))
        hi = int(np.ceil(hi))

        mid, lo, hi = format_with_errorbars(mid, lo, hi)

        results[run_name].setdefault("low", {})[pop] = lo
        results[run_name].setdefault("mid", {})[pop] = mid
        results[run_name].setdefault("high", {})[pop] = hi

14.220903001694502 36.29137411178602 84.26422037747086
1.1678211882111724 6.178323663302014 16.280011220976373
112.62898605723846 258.7007704065045 586.5212332465501
76.33752375313456 176.72913138564286 401.40447798110256
11.882852221599126 30.979862729339093 72.27134606773731
382.1724595582784 867.437073084928 1961.2393670952767


In [64]:
rows = []
for run_name, run_stats in results.items():
    for pop in run_stats["mid"]:
        rows.append(
            {
                "run": run_name,
                "population": pop,
                "low": f"-{run_stats['low'][pop]}",
                "mid": run_stats["mid"][pop],
                "high": f"+{run_stats['high'][pop]}",
            }
        )

results_table = Table(rows=rows, names=("run", "population", "low", "mid", "high"))

In [65]:
results_table

run,population,low,mid,high
str2,str4,str4,str3,str5
O4,BNS,-22,36,+49
O4,NSBH,-5,6,+11
O4,BBH,-150,260,+330
O5,BNS,-100,180,+220
O5,NSBH,-20,31,+42
O5,BBH,-480,870,+1100


In [66]:
# display by run
for run_name in sorted(set(results_table["run"])):
    print(f"\n Observing Run {run_name} : Annual number of detections")
    print("=" * 47)
    print(results_table[results_table["run"] == run_name])
    print("=" * 47)


 Observing Run O4 : Annual number of detections
run population low  mid high
--- ---------- ---- --- ----
 O4        BNS  -22  36  +49
 O4       NSBH   -5   6  +11
 O4        BBH -150 260 +330

 Observing Run O5 : Annual number of detections
run population low  mid  high
--- ---------- ---- --- -----
 O5        BNS -100 180  +220
 O5       NSBH  -20  31   +42
 O5        BBH -480 870 +1100


# For Petrov et al. 2O22 , design as LRR distribustion, 

the BNS , NSBH and BBH as consider as a population not a sub-pospulation from CBC ,

In fact we simulate each population independently to the other ones, so 1 million each.

This means there is no longer a mass fraction or simply means the mass fraction is 1 because as we simulate each population at one time.
This

In [67]:
# Lower 5% and upper 95% quantiles of log-normal distribution for different CBC populations
run_names = ["O4", "O5"]
rates_table = Table(
    [
        # BNS rate from GWTC-2
        # https://doi.org/10.3847/2041-8213/abe949
        {"population": "BNS", "lower": 80.00, "mid": 320.0, "upper": 810.0},
        # NSBH rate from GW200105 and GW200115 paper
        # https://doi.org/10.3847/2041-8213/ac082e
        {"population": "NSBH", "lower": 61.0, "mid": 130.0, "upper": 242.0},
        # BBH rate from GWTC-2
        # https://doi.org/10.3847/2041-8213/abe949
        {"population": "BBH", "lower": 15.3, "mid": 23.9, "upper": 38.2},
    ]
)

rates_table

population,lower,mid,upper
str4,float64,float64,float64
BNS,80.0,320.0,810.0
NSBH,61.0,130.0,242.0
BBH,15.3,23.9,38.2


In [68]:
rates_table["detection_number_O4"] = np.array([1482, 2492, 6040])
rates_table["detection_number_O5"] = np.array([2307, 5441, 21559])

rates_table

population,lower,mid,upper,detection_number_O4,detection_number_O5
str4,float64,float64,float64,int64,int64
BNS,80.0,320.0,810.0,1482,2307
NSBH,61.0,130.0,242.0,2492,5441
BBH,15.3,23.9,38.2,6040,21559


In [69]:
rates_table["sim_rate_O4"] = [
    1.3598513465134647e-05,
    4.462124683568844e-06,
    1.355987556826069e-06,
] * (1 / (u.Mpc**3 * u.yr))
rates_table["sim_rate_O5"] = [
    3.908209488939608e-06,
    1.952463943577305e-06,
    1.0806507866022996e-06,
] * (1 / (u.Mpc**3 * u.yr))


# Conversion in Gpc^-3/ yr
rates_table["sim_rate_O4"] = rates_table["sim_rate_O4"].to(u.Gpc**-3 * u.yr**-1)
rates_table["sim_rate_O5"] = rates_table["sim_rate_O5"].to(u.Gpc**-3 * u.yr**-1)


rates_table["sim_rate_O4"], rates_table["sim_rate_O5"]

(<Column name='sim_rate_O4' dtype='float64' unit='1 / (yr Gpc3)' length=3>
 13598.513465134649
  4462.124683568845
 1355.9875568260693,
 <Column name='sim_rate_O5' dtype='float64' unit='1 / (yr Gpc3)' length=3>
  3908.209488939609
 1952.4639435773054
 1080.6507866022996)

In [70]:
(standard_90pct_interval,) = np.diff(stats.norm.interval(0.9))
rates_table["mu"] = np.log(rates_table["mid"])
rates_table["sigma"] = (
    np.log(rates_table["upper"]) - np.log(rates_table["lower"])
) / standard_90pct_interval

rates_table

population,lower,mid,upper,detection_number_O4,detection_number_O5,sim_rate_O4,sim_rate_O5,mu,sigma
,,,,,,1 / (yr Gpc3),1 / (yr Gpc3),,
str4,float64,float64,float64,int64,int64,float64,float64,float64,float64
BNS,80.0,320.0,810.0,1482,2307,13598.513465134649,3908.209488939609,5.768320995793772,0.7037123471233048
NSBH,61.0,130.0,242.0,2492,5441,4462.124683568845,1952.4639435773054,4.867534450455582,0.41890166985175514
BBH,15.3,23.9,38.2,6040,21559,1355.9875568260693,1080.6507866022996,3.173878458937465,0.2781349878864127


In [71]:
# Extract the log-normal parameters for each population as numpy arrays
fiducial_log_rates = np.asarray(rates_table["mu"])
fiducial_log_rate_errs = np.asarray(rates_table["sigma"])

fiducial_log_rates, fiducial_log_rate_errs

(array([5.768321  , 4.86753445, 3.17387846]),
 array([0.70371235, 0.41890167, 0.27813499]))

In [72]:
rates_table["fiducial_log_rate"] = fiducial_log_rates
rates_table["fiducial_log_rate_err"] = fiducial_log_rate_errs

rates_table

population,lower,mid,upper,detection_number_O4,detection_number_O5,sim_rate_O4,sim_rate_O5,mu,sigma,fiducial_log_rate,fiducial_log_rate_err
,,,,,,1 / (yr Gpc3),1 / (yr Gpc3),,,,
str4,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64
BNS,80.0,320.0,810.0,1482,2307,13598.513465134649,3908.209488939609,5.768320995793772,0.7037123471233048,5.768320995793772,0.7037123471233048
NSBH,61.0,130.0,242.0,2492,5441,4462.124683568845,1952.4639435773054,4.867534450455582,0.41890166985175514,4.867534450455582,0.41890166985175514
BBH,15.3,23.9,38.2,6040,21559,1355.9875568260693,1080.6507866022996,3.173878458937465,0.2781349878864127,3.173878458937465,0.2781349878864127


In [73]:
prob_quantiles = np.asarray([0.05, 0.5, 0.95])
run_duration = 1.0  # years

In [74]:
results = {}
for run_name in run_names:
    results[run_name] = {}

    for pop in ["BNS", "NSBH", "BBH"]:
        rates_row = rates_table[rates_table["population"] == pop]
        rate = rates_row[f"sim_rate_{run_name}"]  # here  rates_row['mass_fraction'] = 1

        mu = (
            rates_row["fiducial_log_rate"]
            + np.log(run_duration)
            + np.log(rates_row[f"detection_number_{run_name}"] / rate)
        )
        sigma = rates_row["fiducial_log_rate_err"]

        lo, mid, hi = poisson_lognormal_rate_quantiles(prob_quantiles, mu, sigma)
        print(lo, mid, hi)
        lo = int(np.floor(lo))
        mid = int(np.round(mid))
        hi = int(np.ceil(hi))

        mid, lo, hi = format_with_errorbars(mid, lo, hi)

        results[run_name].setdefault("low", {})[pop] = lo
        results[run_name].setdefault("mid", {})[pop] = mid
        results[run_name].setdefault("high", {})[pop] = hi

9.330892345522338 34.3640927188856 111.63172036081173
34.05143319981825 72.0894882442789 146.04995420420497
64.03864903155096 105.93977687481468 170.6130780949588
57.7025770477992 188.39270153379962 601.7266733340127
179.43478574585095 361.7728800589275 723.0345185131331
298.3272702572061 476.30088523036954 755.8459665889585


In [75]:
rows = []
for run_name, run_stats in results.items():
    for pop in run_stats["mid"]:
        rows.append(
            {
                "run": run_name,
                "population": pop,
                "low": f"-{run_stats['low'][pop]}",
                "mid": run_stats["mid"][pop],
                "high": f"+{run_stats['high'][pop]}",
            }
        )

results_table = Table(rows=rows, names=("run", "population", "low", "mid", "high"))

In [76]:
results_table

run,population,low,mid,high
str2,str4,str4,str3,str4
O4,BNS,-25,34,+78
O4,NSBH,-38,72,+75
O4,BBH,-42,106,+65
O5,BNS,-130,190,+410
O5,NSBH,-180,360,+360
O5,BBH,-180,480,+280


In [77]:
# display by run
for run_name in sorted(set(results_table["run"])):
    print(f"\n Observing Run  {run_name}")
    print("=" * 30)
    print(results_table[results_table["run"] == run_name])
    print("=" * 30)


 Observing Run  O4
run population low mid high
--- ---------- --- --- ----
 O4        BNS -25  34  +78
 O4       NSBH -38  72  +75
 O4        BBH -42 106  +65

 Observing Run  O5
run population low  mid high
--- ---------- ---- --- ----
 O5        BNS -130 190 +410
 O5       NSBH -180 360 +360
 O5        BBH -180 480 +280
